In [6]:
import numpy as np
import math
import random
#from single_iter_svd import calc_error
#from random import random
#from numpy.random import choice

In [7]:
def columnselect(A,c,k):
    U,S,V = np.linalg.svd(A,full_matrices = False)
    dimension = A.shape
    a = V.shape
  
    p = []
  
    for i in xrange(a[0]):
        p.append(sum(np.dot(V[i,:k+1],V[i,:k+1]))/k)
    
    
    C = np.zeros((dimension[0],c))
    list1 = [i for i in xrange(dimension[1])]
    list2 = [min(1,c*item) for item in p]
    for t in xrange(c):
        
        j = choice(list1)
        #print j
        C[:,t] = A[:,j]/(((c**2) * p[j])**0.5)
    return C

In [8]:
from bisect import bisect
#from random import random

def columnselect2(A,c,k):
    U,S,V = np.linalg.svd(A,full_matrices = False)
    dimension = A.shape
    a = V.shape
    p = []
    p2=[]
    for i in xrange(a[0]):
        p.append(sum((V[i,:k]**2))/k)
    print p
    
    
    C = np.zeros((dimension[0],c))
    #print C.shape
    list2 = [min(1,c*item) for item in p]
    #print list2
    cdf = [list2[0]]
    for i in xrange(1, len(list2)):
        cdf.append(cdf[-1] + list2[i])
    #print cdf
    #print list2
    for t in xrange(c):
        
        j = bisect(cdf,random())
        C[:,t] = A[:,j]/(((c**2) * p[j])**0.5)
    return C

In [38]:
def columnselect3(A,c,flag):
    a = A.shape
    p=[]
    sq_sum=0
    for i in xrange(a[0]):
        for j in xrange(a[1]):
            sq_sum = sq_sum + A[i,j]**2
    C = np.zeros((a[0],c))
    R = np.zeros((r,a[1]))
    random.seed(1)
    if flag == 1:  #columns
        for x in xrange(a[1]):
            sq_sum2 = 0
            for i in xrange(a[0]):
                sq_sum2 = sq_sum2 + A[i,x]**2
            p.append(float(sq_sum2)/float(sq_sum))
        p = sorted(p)
        
        for y in range(1,len(p)):
            p[y] = p[y] + p[y-1]
        #print p
        k=0
        for i in xrange(c):
            
            j = random.random()
            for k in xrange(len(p)):
                if j<p[k]:
                    j = k
                    #print j
                    break
                
            C[:,i] = A[:,j]/((c*p[j])**0.5)
        return C
    
    else: #rows
        for x in xrange(a[0]):
            sq_sum2 = 0
            for i in xrange(a[1]):
                sq_sum2 = sq_sum2 + A[x,i]**2
            p.append(float(sq_sum2)/float(sq_sum))
        p = sorted(p)
        for y in range(1,len(p)):
            p[y] = p[y] + p[y-1]
        #print p
        k=0
        for i in xrange(c):
            #random.seed(1)
            j = random.random()
            for k in xrange(len(p)):
                if j<p[k]:
                    j=k
                    break
            R[i,:] = A[j,:]/((c*p[j])**0.5)
        return R

In [39]:
def calc_error(A,ratings_svd):

    '''
    Caluclating the Frobenius Error
    '''

    error = 0

    for i in range(0,len(A)):
        for j in range(0,len(A[i])):
            error = error + (A[i,j]-ratings_svd[i,j])**2

    error = math.sqrt(error)
    return error

In [40]:
A = np.array([[1,2],[2,3],[3,4]])
c = 2
r = 2
k = 1

C = columnselect3(A,c,1)

B = A.transpose()
R = columnselect3(A,r,0)
U = np.dot(np.linalg.pinv(C),np.dot(A,np.linalg.pinv(R)))
final = np.dot(C,np.dot(U,R))
print calc_error(A,final)
print final

1.17100901807e-14
[[ 1.  2.]
 [ 2.  3.]
 [ 3.  4.]]


In [41]:
input_file = open("ratings.txt","r")
rating_raw = input_file.readlines()
rating_list = []

for line in rating_raw:
    ind_rating = []
    line = line.split(" ")

    user = int(line[0])
    ind_rating.append(user)

    item = int(line[1])
    ind_rating.append(item)

    rating = float(line[2])
    ind_rating.append(rating)	
    rating_list.append(ind_rating)

max_item = 0
max_user = rating_list[len(rating_list)-1][0]
for rating in rating_list:
    if rating[1] > max_item:
        max_item = rating[1]

ratings = np.zeros((max_user, max_item))

for rating in rating_list:
    ratings[rating[0]-1][rating[1]-1] = rating[2]
    
print ratings
print ratings.shape


[[ 2.   4.   3.5 ...,  0.   0.   0. ]
 [ 0.   0.   0.  ...,  0.   0.   0. ]
 [ 0.   0.   0.  ...,  0.   0.   0. ]
 ..., 
 [ 0.   0.   0.  ...,  0.   0.   0. ]
 [ 0.   4.   0.  ...,  0.   0.   0. ]
 [ 1.5  3.   2.  ...,  0.   0.   0. ]]
(1508, 2071)


In [42]:
c = 2000
r = 1500
k = 3



C = columnselect3(ratings,c,1)
#B = ratings.transpose()
R = columnselect3(ratings,r,0)
#R = R.transpose()
U = np.dot(np.linalg.pinv(C),np.dot(ratings,np.linalg.pinv(R)))
final = np.dot(C,np.dot(U,R))
print calc_error(ratings,final)
print final

    #U = np.linalg.pinv(C) * ratings * np.linalg.pinv(R)
#print C*U*R

530.524290102
[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [ -1.45736053e-03  -1.66084163e-01  -7.17579863e-01 ...,  -3.96533469e-01
   -4.75840163e-01  -7.24594228e-01]
 ..., 
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.49999269e+00   2.99916682e+00   1.99640016e+00 ...,   7.87427576e-06
    9.44913091e-06  -1.41458499e-05]]


In [13]:
random.seed(1)
print random.random()
random.seed(1)
print random.random()

0.134364244112
0.134364244112
